<a href="https://colab.research.google.com/github/siahw/speech-recognition/blob/main/Fine_Tune_Wav2Vec2_for_English_ASR_with_%F0%9F%A4%97_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 35.5 MB/s 
     |████████████████████████████████| 596 kB 58.3 MB/s 
     |████████████████████████████████| 895 kB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 50 kB 2.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149864 sha256=4991240e45f6ee5074c1c7284e06fe6c7c8e5f990daa44b111e3b2823ae034aa
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Suc

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
!apt install git-1fs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package git-1fs


In [5]:
#Creating Wav2Vec2CTCTokenizer SECTION which is the tokenizer responsible for decoding the model's predictions.
#so Wav2Vec checkpoint는 linear layer를 더하여 음성과 맞는 단어 의미를 분석에 그 단어의 카테고리로 연결 짖는다.
from datasets import load_dataset,load_metric #loading dataset and look at its structure
timit=load_dataset("timit_asr")

print(timit)

Generating train split:   0%|          | 0/4620 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1680 [00:00<?, ? examples/s]

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/31aa396113a9e0aaf4a6ba07260fd65ff80dfa0d4227fa76a27f1263725ac1b1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})


In [6]:
#It's to remove the unnecessary details and save the rest of the transcribed text for fine tuning.
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [7]:
print(timit)

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 1680
    })
})


In [8]:
#A short function to display some random samples of the dataset and run to get a feeling for the transcriptions.
from datasets import ClassLabel
import random
import pandas as pd #??
from IPython.display import display,HTML #??

def show_random_elements(dataset,num_examples=10):
  assert num_examples <= len(dataset), "Can't pick more elements than there are in dataset." #윗줄에 num_examples=10이라 했는데 왜 이 줄이 필요한지
  picks=[]
  for _ in range(num_examples): #??
    pick=random.randint(0,len(dataset)-1) #?? 왜 len(dataset)-1인지, random integer number between 0 to length of dataset -1
    while pick in picks: #?? 왜 2번하는지
      pick=random.randint(0,len(dataset)-1)
    picks.append(pick)
  df=pd.DataFrame(dataset[picks]) #??
  display(HTML(df.to_html())) #??
show_random_elements(timit["train"].remove_columns(["file","audio"])) #only showing text


,text
0,What targets have we successfully knocked out?
1,Trespassing is forbidden and subject to penalty.
2,A lone star shone in the early evening sky.
3,Stimulating discussions keep students' attention.
4,Continental drift is a geological theory.
5,She had your dark suit in greasy wash water all year.
6,The figure five is important in insurance.
7,So what's this all about?
8,Don't ask me to carry an oily rag like that.
9,While waiting for Chipper she crisscrossed the square many times.


In [9]:
#normalize text to lowercase and also remove the useless .,?! marks because they don't make sound in speech
import re #regular expression module, which has a string related function, like match(),find(),..
chars_to_ignore_regex='[\,\?\.\!\-\;\:\"]'
def remove_special_characters(batch): #?? where this batch come from
  batch["text"]=re.sub(chars_to_ignore_regex,'',batch["text"]).lower() #re.sub('찾을 패턴', '찾은 패턴을 변경할 내용', '원본') 
  return batch
timit=timit.map(remove_special_characters) #?? what is map, It is used when you want to apply a single transformation function to all the iterable elements

  0%|          | 0/4620 [00:00<?, ?ex/s]

  0%|          | 0/1680 [00:00<?, ?ex/s]

In [10]:
show_random_elements(timit["train"].remove_columns(["file", "audio"]))

,text
0,don't look for group valuables in a bank vault
1,the smell is sexual but so powerfully so that a civilized nose must deny it
2,upgrade your status to reflect your wealth
3,a tsunami is not a single wave but a series
4,she had your dark suit in greasy wash water all year
5,diane may splurge and buy a turquoise necklace
6,the advertising verse of plymouth variety store never changes
7,they're easier to shade there
8,the diagnosis was discouraging however he was not overly worried
9,they are both trendfollowing methods


In [11]:
#Connectionist Temporal Classification (CTC) is a type of Neural Network output helpful in tackling sequence problems like handwriting and speech recognition where the timing varies.hey여야 하는데 인식이 hhheyy로 되면 hey로 바꿔준다. Using CTC ensures that one does not need an aligned dataset, which makes the training process more straightforward

In [12]:
#Write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. It is important to pass the argument batched=True to the map(...) function so that the mapping function has access to all transcriptions at once.
def extract_all_chars(batch):
  all_text=" ".join(batch["text"]) #Join in Python is an in-built method used to join an iterable's elements, separated by a string separator, which is specified infront of the join(), which is " "here
  vocab=list(set(all_text)) #Sets are used to store multiple items in a single variable.A set is a collection which is unordered, unchangeable, and unindexed.
  return{"vocab":[vocab],"all_text":[all_text]}
#batch_size는 the number of training examples utilized in one iteration.
vocabs=timit.map(extract_all_chars,batched=True,batch_size=-1,keep_in_memory=True,remove_columns=timit.column_names["train"]) #?? what is keep_in_memory, why does it remove columns train?

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
#Create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.
vocab_list=list(set(vocabs["train"]["vocab"][0])|set(vocabs["test"]["vocab"][0]))
vocab_dict={v:k for k,v in enumerate(vocab_list)} #?? is enumerate() and set() similar but indexed properties? enumerate() allows us to iterate through a sequence but it keeps track of both the index and the element.
vocab_dict

{' ': 5,
 "'": 20,
 'a': 23,
 'b': 16,
 'c': 15,
 'd': 2,
 'e': 8,
 'f': 27,
 'g': 10,
 'h': 19,
 'i': 18,
 'j': 4,
 'k': 21,
 'l': 26,
 'm': 3,
 'n': 14,
 'o': 22,
 'p': 6,
 'q': 11,
 'r': 12,
 's': 1,
 't': 7,
 'u': 17,
 'v': 13,
 'w': 25,
 'x': 0,
 'y': 24,
 'z': 9}

In [14]:
#Note that we did not exclude " "and ' special characters because:
#" ": The model has to learn to predict when a word finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
# ' : In English, we need to keep the ' character to differentiate between words, e.g., "it's" and "its" which have very different meanings.

#But to make " " more visible we change to | instead.
vocab_dict["|"]=vocab_dict[" "] 
del vocab_dict[" "]
#Add "unknown" token so that  the model can later deal with characters not encountered in Timit's training set.
vocab_dict["[UNK]"]=len(vocab_dict)
#Add "padding" token that corresponds to CTC's "blank token".
vocab_dict["[PAD]"]=len(vocab_dict)
print(len(vocab_dict))

30


In [15]:
#Now our vocabulary is complete and consists of 30 tokens, which means that the linear layer that we will add on top of the pretrained Wav2Vec2 checkpoint will have an output dimension of 30.
#?? Drawing explanation please

In [16]:
#Now save the vocabulary as a json file.
import json
with open('vocab.json','w')as vocab_file:
 json.dump(vocab_dict,vocab_file)

In [17]:
#Use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class.
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [18]:
#If you want to re-use the just created tokenizer with the fine-tuned model of this notebook, it is strongly advised to upload the tokenizer to the Hub.
#Call the repo to which we will upload the files "wav2vec2-large-xlsr-turkish-demo-colab".
#(code)repo_name = "wav2vec2-base-timit-demo-colab"
#and upload the tokenizer to the Hub.
#(code)tokenizer.push_to_hub(repo_name)

#It gives error => Looks like you do not have git-lfs installed, please install. You can install from https://git-lfs.github.com/. Then run `git lfs install` (you only have to do this once).

In [19]:
#Create Wav2Vec2 Feature Extractor SECTION

In [20]:
#Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called sampling.연속적인 시그널을 별개의 것으로 구분하다.
#Sampling defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the real speech signal but also necessitates more values per second.

In [21]:
#A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. Thus, before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

In [22]:
#First would have had to up or downsample the speech signal to match the sampling rate of the data used for pretraining.
#So define Wav2Vec2's feature extraction pipeline.
#Feature extraction은 필요한 특징 데이터를 추출해서 사용할 데이터로 정하는 것이다, which refers to the process of transforming raw data into numerical features that can be processed while preserving the information in the original data set. 

In [23]:
from transformers import Wav2Vec2FeatureExtractor
#feature_size: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal
#sampling_rate: The sampling rate at which the model is trained on
#padding_value: Inserting non-informative characters to a string is known as Padding. For batched inference, shorter inputs need to be padded with a specific value. 
#do_normalize: Whether the input should be zero-mean-unit-variance normalized or not. Usually, speech models perform better when normalizing the input
#return_attention_mask: Whether the model should make use of an attention_mask for batched inference. In general, models should always make use of the attention_mask to mask padded tokens. However, due to a very specific design choice of Wav2Vec2's "base" checkpoint, better results are achieved when using no attention_mask. This is not recommended for other speech models.
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [24]:
#To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object.
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer) #?? tokenizer out of nowhere

In [25]:
#Preprocess Data SECTION

In [26]:
#So far, we have not looked at the actual values of the speech signal but just the transcription. In addition to sentence, our datasets include two more column names path and audio. path states the absolute path of the audio file.
print(timit[0]["path"]) #?? error

KeyError: ignored

In [ ]:
#Wav2Vec2 expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.
#Thankfully, datasets does this automatically by calling the other column audio.
common_voice_train[0]["audio"] #error

In [ ]:
#Let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded.
#It can be heard, that the speakers change along with their speaking rate, accent, etc. Overall, the recordings sound relatively clear though, which is to be expected from a read speech corpus.
import IPython.display as ipd
import numpy as np
import random
rand_int = random.randint(0, len(timit["train"])) #random으로 데이터를 불러와서 소리를 내본다
print(timit["train"][rand_int]["text"])
#asarray:np.array() 와 차이점은 이미 ndarray의 데이터 형태 (data type)이 설정 되어 있다면, 데이터 형태가 다를 경우에만 복사(copy) 가 된다.
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000) #?? why asarray?

In [ ]:
#Final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.
rand_int = random.randint(0, len(timit["train"])) #random으로 데이터를 불러와서 출력 해본다
print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape) #1dimensional array여야 하고
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"]) #16kHz여야 한다

In [ ]:
#We can process the dataset to the format expected by the model for training. We will make use of the map( ) function.
def prepare_dataset(batch):
    audio = batch["audio"] #Load and resample the audio data
    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0] #Extract the input_values from the loaded audio file. In our case, the Wav2Vec2Processor only normalizes the data.
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids #Encode the transcriptions(말소리를 음성 기호로 표기) to label ids. 
    return batch

In [ ]:
#Apply the data preparation function to all examples.
#num_proc: num_proc (int, optional, default None) — Number of processes for multiprocessing. By default it doesn’t use multiprocessing.
timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4) #?? what is num_proc is 4

In [ ]:
#Training & Evaluation SECTION
#The data is processed so that we are ready to start setting up the training pipeline. 

In [ ]:
#Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample.
#This data collator treats the input_values and labels differently and thus applies to separate padding functions on them, because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
@dataclass
class DataCollatorCTCWithPadding:
  processor: Wav2Vec2Processor #processor (:class:): The processor used for proccessing the data.
  padding: Union[bool, str] = True # padding (:obj:`bool`, :obj:`str` or :class:, :obj:`True`): Select a strategy to pad the returned sequences according to the model's padding side and padding index 
  #:obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch or no padding if only a single sequence if provided.
  #:obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the maximum acceptable input length for the model if that argument is not provided.
  #:obj:`False` or :obj:`'do_not_pad'` (default): No padding. It can output a batch with sequences of different lengths.
  max_length: Optional[int] = None #max_length (:obj:`int`, `optional`): Maximum length of the ``input_values`` of the returned list and optionally padding length 
  max_length_labels: Optional[int] = None #max_length_labels (:obj:`int`, `optional`): Maximum length of the ``labels`` returned list and optionally padding length
  pad_to_multiple_of: Optional[int] = None  #pad_to_multiple_of (:obj:`int`, `optional`): If set will pad the sequence to a multiple of the provided value.
  pad_to_multiple_of_labels: Optional[int] = None

In [ ]:
def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]: #?? why torch.Tenson out of nowhere
 #Split inputs and labels since they have to be of different lenghts and need different padding methods
  input_features = [{"input_values": feature["input_values"]} for feature in features]
  label_features = [{"input_ids": feature["labels"]} for feature in features]
  batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
  with self.processor.as_target_processor():
    labels_batch = self.processor.pad(
      label_features,
      padding=self.padding,
      max_length=self.max_length_labels,
      pad_to_multiple_of=self.pad_to_multiple_of_labels,
      return_tensors="pt",
        )
#Analogous to the common data collators, the padding tokens in the labels with -100 so that those tokens are not taken into account when computing the loss, which means # replace padding with -100 to ignore loss correctly.
  labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
  batch["labels"] = labels
  return batch

In [ ]:
#Initialize the data collator.
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")#The predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1) #We are interested in the most likely prediction of the model and thus take the argmax(...) of the logits. 
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id #?? 무슨의미인지 Transform the encoded labels back to the original string by replacing -100 with the pad_token_id and decoding the ids 
    pred_str = processor.batch_decode(pred_ids) #?? 무슨의미인지
    # we do not want to group tokens when computing the metrics, making sure that consecutive tokens are not grouped to the same token in CTC style
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False) #?? what is the reason. To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model
    #If not False for group_tokens, a word like "hello" would incorrectly be encoded, and decoded as "helo".
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [ ]:
#Load the pretrained Wav2Vec2 checkpoint. 
from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained( #?? 어떤 의미인지
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", #To save GPU memory, we enable PyTorch's gradient checkpointing and also set the loss reduction to "mean".
    pad_token_id=processor.tokenizer.pad_token_id, #The tokenizer's pad_token_id must be to define the model's pad_token_id or in the case of Wav2Vec2ForCTC also CTC's blank token
)

In [ ]:
model.freeze_feature_extractor() #?? 어떤 의미인지

In [ ]:
#Define all parameters related to training.

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
  output_dir=repo_name, #output_dir (str): The output directory where the model predictions and checkpoints will be written.
  group_by_length=True, #group_by_length makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model. group_by_length (bool, optional, defaults to False): Whether or not to group together samples of roughly the same length in the training dataset (to minimize padding applied and be more efficient). Only useful if applying dynamic padding.
  per_device_train_batch_size=32, #per_device_train_batch_size (int, optional, defaults to 8): The batch size per GPU/TPU core/CPU for training.
  evaluation_strategy="steps", #evaluation_strategy (str or IntervalStrategy, optional, defaults to "no"): The evaluation strategy to adopt during training.
  #Possible values are:
  # "no": No evaluation is done during training.
  # "steps": Evaluation is done (and logged) every eval_steps.
  # "epoch": Evaluation is done at the end of each epoch.
  num_train_epochs=30, #num_train_epochs(float, optional, defaults to 3.0): Total number of training epochs to perform, if not an integer, will perform the decimal part percents of the last epoch before stopping training.
  fp16=False, #fp16 (bool, optional, defaults to False): Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.빠르게 하려고 16bit
  gradient_checkpointing=True, #gradient_checkpointing (bool, optional, defaults to False): If True, use gradient checkpointing to save memory at the expense of slower backward pass.
  save_steps=500, #save_steps (int, optional, defaults to 500): Number of updates steps before two checkpoint saves if save_strategy="steps".
  eval_steps=500, #eval_steps (int, optional): Number of update steps between two evaluations if evaluation_strategy="steps". Will default to the same value as logging_steps if not set.
  logging_steps=500, #logging_steps (int, optional, defaults to 500): Number of update steps between two logs if logging_strategy="steps".
  #learning_rate and weight_decay were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.
  learning_rate=1e-4,#learning_rate (float, optional, defaults to 5e-5): The initial learning rate for AdamW optimizer
  weight_decay=0.005,#weight_decay (float, optional, defaults to 0): The weight decay to apply (if not zero) to all layers except all bias and LayerNorm weights in AdamW optimizer.
  warmup_steps=1000, #warmup_steps (int, optional, defaults to 0): Number of steps used for a linear warmup from 0 to learning_rate. Overrides any effect of warmup_ratio.
  save_total_limit=2, #?? error  save_total_limit (int, optional): If a value is passed, will limit the total amount of checkpoints. Deletes the older checkpoints in output_dir.
)

In [ ]:
#All instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer
trainer = Trainer( #?? 왜 같은 단어를 다 = 하는지
    model=model,
    data_collator=data_collator,
    args=training_args, #?? error
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
#Training SECTION

In [ ]:
#In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab and insert code).
#function ConnectButton(){
#    console.log("Connect pushed"); 
#    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
#}
#setInterval(ConnectButton,60000);

In [ ]:
trainer.train()

In [ ]:
#Evaluation SECTION
#Evaluate our fine-tuned model on the test set and play around with it a bit.

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)

In [ ]:
#Make use of the map( ) function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary "results".

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0) #?? why unsqueeze
    logits = model(input_values).logits #?? what is logits
  pred_ids = torch.argmax(logits, dim=-1) 
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  return batch
results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

In [ ]:
#compute the overall WER
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

In [ ]:
#Some predictions to see what errors are made by the model.
show_random_elements(results.remove_columns(["speech", "sampling_rate"]))

In [ ]:
#To better understand how CTC works, it is worth taking a deeper look at the exact output of the model. Let's run the first test sample through the model, take the predicted ids and convert them to their corresponding tokens. The model is to some extent invariant to speaking rate since it has learned to either just repeat the same token in case the speech chunk to be classified still corresponds to the same token. This makes CTC a very powerful algorithm for speech recognition since the speech file's transcription is often very much independent of its length.
model.to("cuda")
with torch.no_grad():
  logits = model(torch.tensor(timit["test"][:1]["input_values"], device="cuda")).logits
pred_ids = torch.argmax(logits, dim=-1)
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist())) #convert ids to tokens